#Coords and Emissions
The goal of this project is to take methane and CO2 emissions JSON file from carbon mapper, and use our detection models to evaluate what structures they are

The pipeline should go as follows
1. Download JSON file from carbon mapper (dont need to automate now)
2. Break down file into a list containing coordinates and emissions rate
3. Take image of each coordinate (do this all in one go)
4. Loop through each image with compressor station and pad models (serially, not in parallal)
5. Use a basic algorithm to automatically make a decision on the objects identity
    - Note, we dont have a method for processing facilities, but the CS model may detect them anyways
    - Tank batteries and pads are likley wells, with seperators are almost certainly pads. Once A.L is done we can diff. wells and gathering facilities

In [2]:
from google.colab import drive
import os
drive.mount('/content/gdrive') # only if using google colab

root = '/content/gdrive/My Drive/Wurk/SURI/Coords and emissions' #Path where this notebook is located
os.chdir(root)

Mounted at /content/gdrive


In [7]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' # installs detecron2, be warned this does take a bit (~5-10 minutes)

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-m9t9v7y5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-m9t9v7y5
  Resolved https://github.com/facebookresearch/detectron2.git to commit 18f69583391e5040043ca4f4bebd2c60f0ebfde0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.2 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6052954 sha256=86cf5874b011c914ab0d0694301566b7550fc8b5b9f4d64c6a9f6c10632a81f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ep2z_3_f/wheels/17/d9/40/60db98e485aa9455d

In [3]:
!python -m pip install folium
!python -m pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0


In [8]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

#Run this if using detectron2

In [4]:
'''
Note: Not all of these modules are needed, delete the unecessary ones
'''

from google.colab.patches import cv2_imshow

#Imports for model evaluation
import cv2

#imports for 486

import numpy as np
import os
import tempfile
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import folium
import time
from PIL import Image

#imports for 486 test
import random #need a simulated environment to test coords w/o using AI all the time

#Imports for search area creation
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import math
from shapely.geometry import Polygon, box

#Imports for JSON reading
import json
import matplotlib.pyplot as plt
import numpy as np

import json
from datetime import datetime
import re
import ast


## Necessary functions
We will need these functions for this project. For some reason Google colab makes it difficult to call in modules from other files. Once I figure out how to make Detectron2 work locally this will all be in a .py structure

In [9]:
def generate_coords(diag_pt1, diag_pt2, num_points):
    """
    Generate a list of equally spaced coordinates within a square
    defined by two diagonal points.

    Parameters:
        diag_pt1 (tuple): (x1, y1) - one corner of the square
        diag_pt2 (tuple): (x2, y2) - opposite corner of the square
        num_points (int): number of coordinates to generate

    Returns:
        list of (x, y) tuples
    """

    # Get the bounds
    x_min, x_max = sorted([diag_pt1[0], diag_pt2[0]])
    y_min, y_max = sorted([diag_pt1[1], diag_pt2[1]])

    # Calculate number of points per side to form a grid
    side_points = int(np.ceil(np.sqrt(num_points)))

    # Generate a grid of x and y values
    x_vals = np.linspace(x_min, x_max, side_points)
    y_vals = np.linspace(y_min, y_max, side_points)

    coords = [(x, y) for x in x_vals for y in y_vals]

    # Trim to exactly `num_points`
    return coords[:num_points]


def launch_driver():
    '''
    When writing this, there was a reucurring issue with the driver, this function fixed it, so we need to call it only once, idk why

    returns:
    driver - object

    '''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=800,800")
    chrome_options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")
    chrome_options.add_argument(f"--remote-debugging-port=9222")
    return webdriver.Chrome(options=chrome_options)


def master_coords(coordinates, directory):
    '''
    Takes a list of coordinates and returns images of them

    Parameters:

    list of tuples of coordinates [(lat, long), (lat, long), ....]
    Latitude and longitude are in decimal system with positives and negatives, not degress/minutes/seconds system

    directory - file where you want your images sent to

    NOTE: This function is a major time bottlneck for 486, because it uses a web driver which physically navigates Esri's imagery database
    Once I integrate sentinel (soon), this could cut down 486's operation time signficantly

    Similarly, because this uses a webdriver, artifacts from the webpage may exist on the output images (watermarks, zoom buttons)
    '''



    zoom = 18 #how much were zooming in
    output_dir = os.path.join(root, directory) # directory were saving images to
    os.makedirs(output_dir, exist_ok=True) #actually making this directory

    driver = launch_driver()  # Launch once

    # === Generate map HTML with Esri imagery ===
    def generate_satellite_map(lat, lon, zoom, html_filename):
        m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles=None) #folium map

        # Add Esri World Imagery
        folium.TileLayer(
            tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
            attr="Esri",
            name="Esri Satellite",
            overlay=False,
            control=False,
        ).add_to(m)

        m.save(html_filename)

    '''
    The previous function creates an html file with all the mapping stuff. This is not usable for the AI. It must be converted to JPEG
    '''


    def html_to_jpeg(html_file, jpeg_output_file):
        driver.get("file://" + os.path.abspath(html_file))
        time.sleep(2)  # wait for map tiles to load

        temp_png = jpeg_output_file.replace(".jpg", "_temp.png")
        driver.save_screenshot(temp_png)

        with Image.open(temp_png) as im:
            rgb_im = im.convert("RGB")
            rgb_im.save(jpeg_output_file, "JPEG", quality=95)

        os.remove(temp_png)

    '''
    This whole process takes a long time, and each images takes a few seconds to generate, this means only small areas should be analyzed to stay in computation limits
    '''

    '''
    Below, we loop through each coordinate, generating an image, we also delete the html file
    '''
    # === Run for each coordinate ===
    for idx, (lat, lon) in enumerate(coordinates):
        html_file = f"temp_map_{idx}.html"
        jpeg_file = os.path.join(output_dir, f"satellite_{idx+1}_{lat}_{lon}.jpg")

        generate_satellite_map(lat, lon, zoom, html_file)
        html_to_jpeg(html_file, jpeg_file)

        print(f"Saved JPEG: {jpeg_file}")
        os.remove(html_file)  # optional: clean up

    driver.quit()

## Carbonmapper related code
This section takes the plumes.json file and turns it into a dictionary for our purposes

In [10]:
# Replace with your actual JSON file path
# make this a function later

def plume_data(directory, percentile):
  file_path = os.path.join(root, directory) #put the name of the JSON file

  # Open and read the JSON file
  with open(file_path, "r") as f:
      data = json.load(f)


  xx = []
  yy = []
  coord_list = []
  emission_list = []

  #grabbing data

  for i in data['features']:
      coord = i['geometry']['coordinates']
      xx.append(coord[0])
      yy.append(coord[1])
      coord_list.append((coord[1], coord[0]))
      emission = i['properties']['emission_auto']
      if emission is not None:
          emission_list.append(emission)

  '''
  Here, we take a threshold of emitters (the total, number is in the thousands), and find the coordinates and emission amount of them
  '''
  threshold = np.percentile(emission_list, percentile)

  # Step 2: Get indices of emissions > threshold
  emission_list = np.array(emission_list)
  top_indices = np.where(emission_list > threshold)[0]

  # Step 3: Get top emissions and corresponding coordinates
  top_emissions = emission_list[top_indices]
  top_xx = [xx[i] for i in top_indices]
  top_yy = [yy[i] for i in top_indices]

  top_emitters = {}
  for j in range(len(top_emissions)):
      top_emitters[(top_yy[j], top_xx[j])] = top_emissions[j]

  print(top_emitters)

  plt.figure(figsize=(6, 6))
  plt.scatter(top_xx, top_yy, c='red', marker='o')
  plt.title("Top Emitter Coordinates")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  plt.grid(True)
  plt.axis('equal')  # Keeps aspect ratio consistent
  plt.show()

  #top_emitters_sample = dict(random.sample(list(top_emitters.items()), 10))
  return top_emitters


## SIC Related Code

### Base SIC function

In [16]:
def emission_ident(emission_directory, percentile, model_name, directory, tolerance, outfilename, num_classes):

  input_path = os.path.join(root, "inputs", emission_directory)
  e = input_path
  emission_dict = plume_data(e, percentile)

  coordinates = []
  for i in emission_dict:
    coordinates.append(i)

  print("coordinates generated")
  size = len(coordinates)
  print(f"There are {size} to process (estimated {size * 2 / 60} minutes to process)")

  master_coords(coordinates, directory)

  print("Images generated")

  from detectron2.engine import DefaultPredictor

  print(f"Calling in model {model_name}")
  cfg = get_cfg()
  cfg.MODEL.DEVICE = "cpu"
  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)  # path to the model being used (this may change in the future)
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = tolerance   # set a custom testing threshold
  predictor = DefaultPredictor(cfg)
  print(f"{model_name} called\n")


  files_path_eval = os.path.join(root, directory) # the directory were using
  files_eval = os.listdir(files_path_eval) #makes a list of files in the directory, each one will be called in the for loop

  ### We dont need a review fila as all files will be datamined

  #files_for_review = os.path.join(root, f"{directory} REVIEW FOLDER")
  #os.makedirs(files_for_review, exist_ok=True)


  print("Begining file evaluation\n")

  main_dict = {}

  '''
  for i in master_dictionary:
      if master_dictionary[i] == "HIGH":
          i = i.strip(".jgp")
          i = i.split("_")
          results.append((float(i[2]), float(i[3])))

  '''


  for file in files_eval:
    print(f"################################Evaluating {file}######################################### \n\n\n")
    site_dict = {}
    path = os.path.join(files_path_eval, file)
    im = cv2.imread(path) #puts the file into the image interface
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                    scale=3.0,
                    instance_mode=ColorMode.IMAGE_BW
    )

    save_path = os.path.join(files_for_review, file) #we must always datamine

    # Get predictions
    instances = outputs["instances"]
    pred_classes = instances.pred_classes.cpu().tolist()  # class indices
    scores = instances.scores.cpu().tolist()              # confidence scores

    #Visualize the image
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

    # Map class indices to labels
    class_names = ["tank battery", "well pad", "artificial lift", "separator", "compressor"]
    labels = [class_names[i] for i in pred_classes]

      # Count each object type
    from collections import Counter
    counts = Counter(labels)

    # Print scores and counts
    for label, score in zip(labels, scores):
        print(f"Detected: {label} with confidence {score:.2f}")
        if label not in site_dict:
          site_dict[label] = [score]
        else:
          site_dict[label].append(score)

    #filename contains coord data, we extract this to obtain emissions amount, as well as pass to the main dictionary for the output
    file_coord = file
    file_coord = file_coord.strip(".jgp")
    file_coord = file_coord.split("_")
    site_dict["emission"] = emission_dict[(float(file_coord[2]), float(file_coord[3]))] #The key for the emission amount is the coordinates

    main_dict[((float(file_coord[2]), float(file_coord[3])))] = site_dict

    '''
    Site_sort (see code below) allows us to not have to review every single image. Some combinates of scores and features
    are almost always indicative of a actual O&G facility, and are automatically marked as positives

    On the other hand, some sites (low confidence, singular pads) are almost never a useful site, and are thrown out of the data

    This sorting is very important, and with more data, can be expanded to make manual review only necessary for extreme cases

    Some ideas could include:
    - - - Looking at the overlap of detection masks
    - - -

    '''

    sort_tag = site_sort(site_dict)

    if sort_tag == None:
      site_dict['status'] = "Check"
      print(f"No Facility could be determined\n - We record {site_dict}")
      main_dict[((float(file_coord[2]), float(file_coord[3])))] = site_dict



    elif sort_tag == "BAD":
      print("Only Low Confidence Pad Dectected, likley not a site")
      site_dict['status'] = 'bad'
      print(f"\nFull Site Dict \n{site_dict}\n")
      os.remove(save_path)
      #Removes bad data


    else:
      site_dict['status'] = 'good'
      site_dict['Facility Type'] = sort_tag
      print(f"{site_dict['Facility Type']} Detected")
      print(f"\nFull Site Dict \n{site_dict}\n")
      main_dict[((float(file_coord[2]), float(file_coord[3])))] = site_dict

    print(f"File {file} analyzed \n\n\n")

  print("Files completed, make sure to sort files based on actual facility, and sent mined images to datatorch")

  #Note: We arent doing any cleaning with this program as the files need to be datamined

  # Get current date and time formatted for filename

  timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  filename = f"{outfilename}_{timestamp}.json"

  save_path = os.path.join(root, "exports")
  os.makedirs(save_path, exist_ok=True)
  save = os.path.join(save_path, f"{filename}")

  # Write dictionary to JSON file
  with open(save, "w") as json_file:
      json.dump(str(main_dict), json_file, indent=4)

  print(f"Dictionary saved to {filename}")


  return main_dict




In [23]:
emission_ident("canada_715.json", 0, "tb_model_v3.pth", "CM Canada 715 1", 0.5, "CM Canada 715 1 dict", 5)

Output hidden; open in https://colab.research.google.com to view.

In [30]:
emission_ident("bakken 715.json", 0, "tb_model_v3.pth", "CM Bakken 715 1", 0.5, "CM Bakken 715 1 dict", 5)

Output hidden; open in https://colab.research.google.com to view.

### Sorting Function
Make sure this mirrors SIC's site sort. We dont need manual review as all sites are positives inherently.

In [24]:
#class_names = ["tank battery", "well pad", "artificial lift", "separator", "compressor"]

def site_sort(site_dict):

    tag = []
    tank_val = site_dict.get('tank battery', 0)
    sep_val = site_dict.get('separator', 0)
    lift_val = site_dict.get('artificial lift', 0)
    comp_val = site_dict.get('compressor', 0)
    pad_val = site_dict.get('well pad', 0)

    #Case 0: Processing Facility

    if 'separator' in site_dict and 'compressor' in site_dict:
        if any(x > 0.5 for x in site_dict['separator']) and len(site_dict['compressor']) > 2:
            tag.append("Processing facility ")

    # Case 1: Compressor Station
    elif 'well pad' in site_dict and 'compressor' in site_dict:
        if any(x > 0.5 for x in site_dict['well pad']) and len(site_dict['compressor']) > 1:
            tag.append("Compressor Station")

    # Case 2: Upstream Facility
    elif 'tank battery' in site_dict or 'separator' in site_dict:
        if 'well pad' in site_dict and 'tank battery' in site_dict:
            if any(x > 0.6 for x in site_dict['tank battery']) and any(x > 0.8 for x in site_dict['well pad']):
              #a ton of false positivites with tbs and seps, need a good pad threshhold
              tag.append("Upstream Facility")
        if 'well pad' in site_dict and 'separator' in site_dict:
            if any(x > 0.8 for x in site_dict['well pad']) and any(x > 0.5 for x in site_dict['separator']):
              tag.append("Upstream Facility")

    # Case 3: Well Pad
    if 'artificial lift' in site_dict and 'well pad' in site_dict:
        if any(x > 0.5 for x in site_dict['artificial lift']) and any(x > 0.7 for x in site_dict['well pad']): #If theres a pump jack and somthing like a pad, its def a ped
            tag.append("Well Pad with Artificial Lift")

    elif tank_val == 0 and sep_val == 0 and comp_val == 0 and lift_val == 0: #Everything else needs to be zero to be sure
        if 'well pad' in site_dict and len(pad_val) == 1:
            if any(x > 0.90 for x in site_dict['well pad']):
                print("High Confidence Well Pad")
                tag = "General Well Pad"

    #Unlike SIC, there are no false positives, instead we must label the tag

    if tag == []:
      tag = input("No facility identified, please input a tag: ")


    return tag





# Debreif notes 7/1/25

The most recent facility model has promising results. Because of the nature of the data, false positives not In oil and gas are largley prevented. Intra-O&G false positives are an issue, but the nature of data collection allows a very good opportunity for datamining.

Some possible next steps:
1. Continue training of the upstream facility model (UFM)
    - Label new images from datamining
    - Create 2-3 new labels (finish aritificial lift)
    - ***Note - it may be promising to create a model that detects compressors, which could help the stalled C.S Identification project.

In the future, we can match this data with databases to find operator

Our JSON file returns a string, and it must be covnerted (will fix this soon)

## Exporting Functions
Keep these here in case you need to export data

In [5]:
def strip_np_float64(text):
    # Replace np.float64(123.456) → 123.456
    return re.sub(r'np\.float64\(([^()]+)\)', r'\1', text)

def load_custom_dict(filepath):
    with open(filepath, 'r') as f:
        raw = f.read().strip()

    # Remove outer quotes if present
    if raw.startswith('"') and raw.endswith('"'):
        raw = raw[1:-1]

    # Strip np.float64() calls
    cleaned = strip_np_float64(raw)

    # Now parse it safely
    data = ast.literal_eval(cleaned)
    return data



In [6]:
file_path = os.path.join(root, 'exports', 'CM Permian 714 1 dict_2025-07-14_22-31-19.json')
load_custom_dict(file_path)

{(31.827120263737015, -101.9069071781443): {'well pad': [0.9271409511566162],
  'compressor': [0.8982704281806946,
   0.8907779455184937,
   0.8872802257537842,
   0.829702615737915,
   0.7934798002243042,
   0.7891572117805481,
   0.7813464999198914,
   0.7565206289291382,
   0.6646436452865601],
  'tank battery': [0.8814815878868103],
  'emission': 361.83384451724737,
  'status': 'good',
  'Facility Type': ['Compressor Station']},
 (31.586059999999982, -101.88388299999998): {'well pad': [0.969664454460144],
  'emission': 166.52994087500002,
  'status': 'good',
  'Facility Type': 'General Well Pad'},
 (31.849398956852426,
  -101.82053786535154): {'tank battery': [0.9565179347991943,
   0.9281898736953735], 'well pad': [0.8858121037483215], 'artificial lift': [0.6093114018440247,
   0.6061576008796692], 'emission': 185.9474761575131, 'status': 'good', 'Facility Type': ['Upstream Facility',
   'Well Pad with Artificial Lift']},
 (31.852208704095858,
  -101.7765852594473): {'tank battery